# Dog Breed Classification based on RESNET-50(CNN)

Whole project is designed using the following steps:<br><br>
<ol>1. Perform EDA to know the Data</ol>
<ol>2. Handling unbalance dataset</ol>
<ol>3. Creation of the Model</ol>
<ol>4. Training the model</ol>
<ol>5. Test the model</ol>
<ol>6. Calculate the model metrics</ol>
<ol>7. Prediction</ol>

In [1]:
# base packages
import pandas as pd
import os
from shutil import copyfile
import numpy as np

# for split dataset into train, test & validation
import splitfolders

# for creating model
from tensorflow.keras.applications import ResNet50
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense
from tensorflow.keras import optimizers

# for prevention of model overfit
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint

# for data generation
from keras.applications.resnet50 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator

# for testing
from keras.applications.imagenet_utils import preprocess_input
from keras.models import load_model
from keras.preprocessing import image

# for model scoring & metric
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import roc_curve, auc, roc_auc_score

# Read data file

In [2]:
# read the data from the CSV
labels_data = pd.read_csv("labels.csv")

In [3]:
labels_data.describe()

,id,breed
count,10222,10222
unique,10222,120
top,d265584fd8255cf0ffb477cdfddd32f9,scottish_deerhound
freq,1,126


# Configuration

In [4]:
# set base path
TRAIN_DATA = "./train/"
INPUT_PATH = "./temporary_input_data/"
OUTPUT_DATASET = "./input_data"
TESING_DATASET = "./input_data/test"

In [5]:
POOLING = "avg"
WEIGHTS = "imagenet"

CLASSES = None
DENSE_ACTIVATION = "softmax"

OBJ_FUNCTION = "categorical_crossentropy"
LOSS_METRICS = ['accuracy']

EARLY_STOP_PATIENCE = 3

IMAGE_SIZE = 224
TRAINING_BATCH_SIZE = 100

STEPS_PER_EPOCH_TRAINING = 10
NUM_EPOCHS = 10
STEPS_PER_EPOCH_VALIDATION = 10

TESTING_BATCH_SIZE = 100


<U><B>Point No. 1</B></U><br>
According to the statement <B>"The classifier should only predict scores for these breeds : beagle, chihuahua, doberman, french_bulldog, golden_retriever, malamute, pug, saint_bernard, scottish_deerhound, tibetan_mastiff."</B>
<br>
So we are creating a list of the dogs name.

In [6]:
selected_labels = [
    "beagle", 
    "chihuahua", 
    "doberman", 
    "french_bulldog", 
    "golden_retriever", 
    "malamute", 
    "pug", 
    "saint_bernard", 
    "scottish_deerhound", 
    "tibetan_mastiff"]

CLASSES = len(selected_labels)

## 1. Perform EDA

In [7]:
# check the distribution of each label
labels_data[labels_data.breed.isin(selected_labels)].groupby(["breed"]).agg(len).T.to_dict("r")[0]

/home/aniket/.local/lib/python3.6/site-packages/pandas/core/frame.py:1490: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  FutureWarning,


{'beagle': 105,
 'chihuahua': 71,
 'doberman': 74,
 'french_bulldog': 70,
 'golden_retriever': 67,
 'malamute': 81,
 'pug': 94,
 'saint_bernard': 84,
 'scottish_deerhound': 126,
 'tibetan_mastiff': 69}

In [8]:
# get the minimum count from the list
minimum_count = labels_data[labels_data.breed.isin(selected_labels)].groupby(["breed"]).agg(len).min()

## 2. Handling Unbalanced Dataset

In [9]:
# iterate each of the labels from above list
for each_label in selected_labels:
    
    # list all the images name have the present label
    images = list(labels_data[labels_data["breed"] == each_label].head(minimum_count.id).id)
    
    # iterate each of the image from the list
    for image_id in images:
        
        # set the source image 
        image_src = TRAIN_DATA + image_id + ".jpg"
        
        # set the destination path
        image_dst = INPUT_PATH + each_label + "/"
        
        # check if folder not exist with label name
        if not os.path.exists(image_dst):
            
            # create the folder
            os.makedirs(image_dst)
        
        # copy the image into the folder
        copyfile(image_src, image_dst + "/" + image_id + ".jpg")

In [10]:
# split dataset into train, test & validation
splitfolders.ratio(INPUT_PATH, OUTPUT_DATASET, seed=42, ratio=(.6, .2, .2))

Copying files: 670 files [00:00, 4591.47 files/s]


## 3. Model creation

<U><B>Point No. 3</B></U><br>
The classifier should only be built using <B>Resnet50</B> CNN architecture.

In [28]:
# create a Sequential object
model = Sequential()

# add ResNet50 arch into the layer array
model.add(ResNet50(include_top = True, pooling = POOLING, weights = WEIGHTS))

# add prediction layer
model.add(Dense(CLASSES, activation = DENSE_ACTIVATION))

# customize trainable parameter
model.layers[0].trainable = False

In [12]:
# view model summary
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 1000)              25636712  
_________________________________________________________________
dense (Dense)                (None, 10)                10010     
Total params: 25,646,722
Trainable params: 10,010
Non-trainable params: 25,636,712
_________________________________________________________________


In [13]:
# use Stochastic Gradient Descent as Optimizer
sgd = optimizers.SGD(lr = 0.01, decay = 1e-6, momentum = 0.9, nesterov = True)

In [14]:
# compile the model
model.compile(optimizer = sgd, loss = OBJ_FUNCTION, metrics = LOSS_METRICS)

## 4. Model Training

In [15]:
# normalize the data
data_generator = ImageDataGenerator(preprocessing_function=preprocess_input)

In [16]:
# create training data
train_generator = data_generator.flow_from_directory(
    './input_data/train',
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=TRAINING_BATCH_SIZE,
    class_mode='categorical')

Found 400 images belonging to 10 classes.


In [17]:
# create validation data
validation_generator = data_generator.flow_from_directory(
    './input_data/val',
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=TRAINING_BATCH_SIZE,
    class_mode='categorical')

Found 130 images belonging to 10 classes.


In [18]:
# create early_stopper to prevent overfit.
early_stopper = EarlyStopping(monitor = 'val_loss', patience = EARLY_STOP_PATIENCE)

# create checkpointer for store the model into a file
checkpointer = ModelCheckpoint(filepath = 'new_model.hdf5', monitor = 'val_loss', save_best_only = True, mode = 'auto')

In [19]:
# model training & storing
fit_history = model.fit_generator(
        train_generator,
        steps_per_epoch=STEPS_PER_EPOCH_TRAINING,
        epochs = NUM_EPOCHS,
        validation_data=validation_generator,
        validation_steps=STEPS_PER_EPOCH_VALIDATION,
        callbacks=[checkpointer, early_stopper]
)

/home/aniket/.local/lib/python3.6/site-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
10/10 [==============================] - 67s 6s/step - loss: 2.2776 - accuracy: 0.2232 - val_loss: 2.2747 - val_accuracy: 0.2462


In [20]:
# load training checkpoint
model.load_weights("./new_model.hdf5")

In [21]:
# show the training metrics
fit_history.history

{'loss': [2.2764453887939453],
 'accuracy': [0.23499999940395355],
 'val_loss': [2.2746665477752686],
 'val_accuracy': [0.2461538463830948]}

## 5. Model testing

In [22]:
# create a test object
test_generator = data_generator.flow_from_directory(
    directory = TESING_DATASET,
    target_size = (IMAGE_SIZE, IMAGE_SIZE),
    batch_size = TESTING_BATCH_SIZE,
    class_mode = None,
    shuffle = False,
    seed = 123
)

test_generator.reset()

Found 140 images belonging to 10 classes.


In [23]:
# predict the test data
prediction = model.predict_generator(test_generator, steps = len(test_generator), verbose = 1)
predicted_class_indices = np.argmax(prediction, axis = 1)

/home/aniket/.local/lib/python3.6/site-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


2/2 [==============================] - 19s 5s/step


## 6. Model Metrics

<U><B>Point No. 4</B></U><br>
Evaluation metrics i.e <B>Accuracy, Confusion Matrix, F1 Score, ROC-AUC Score</B> shall be calculated
on test data.

In [24]:
y_pred = np.argmax(prediction, axis=1)
print('Confusion Matrix')
print(confusion_matrix(test_generator.classes, y_pred))

print('Classification Report')
print(classification_report(test_generator.classes, y_pred, target_names=selected_labels))

print("ROC-AUC Score")
print(roc_auc_score(y_pred, prediction, multi_class='ovr'))

Confusion Matrix
[[ 1  0  0  0  0  0  0 12  0  1]
 [ 1  0  1  0  0  0  9  2  0  1]
 [ 1  2  0  0  0  0 10  1  0  0]
 [ 0  0  1  2  0  9  0  0  1  1]
 [ 2  0  0  0  3  1  0  8  0  0]
 [ 1  0  0  0  0  3  0  3  7  0]
 [ 0  0  0  0  0 13  1  0  0  0]
 [ 0  0  0  0  0  0  0 14  0  0]
 [ 1  0  0  0  2  0  0  0 11  0]
 [ 1  0  0  0  9  0  0  1  0  3]]
Classification Report
                    precision    recall  f1-score   support

            beagle       0.12      0.07      0.09        14
         chihuahua       0.00      0.00      0.00        14
          doberman       0.00      0.00      0.00        14
    french_bulldog       1.00      0.14      0.25        14
  golden_retriever       0.21      0.21      0.21        14
          malamute       0.12      0.21      0.15        14
               pug       0.05      0.07      0.06        14
     saint_bernard       0.34      1.00      0.51        14
scottish_deerhound       0.58      0.79      0.67        14
   tibetan_mastiff       0.50

## 7. Prediction

In [25]:
# set image path for testing
image_path = "./test/1c16315c9efe0ea92a38cdd20aa9d624.jpg"

# load image
original = image.load_img(image_path, target_size=(224, 224))

# convert into numpy array
numpy_image = image.img_to_array(original)

# expand dimension
image_batch = np.expand_dims(numpy_image, axis=0)

# normalized image
processed_image = preprocess_input(image_batch, mode='tf')

# predict the image
preds = model.predict(processed_image)

In [26]:
dict(zip(selected_labels, preds[0]))

{'beagle': 0.099908516,
 'chihuahua': 0.09955728,
 'doberman': 0.099856965,
 'french_bulldog': 0.09963106,
 'golden_retriever': 0.10047741,
 'malamute': 0.099312104,
 'pug': 0.1002064,
 'saint_bernard': 0.09981777,
 'scottish_deerhound': 0.100539885,
 'tibetan_mastiff': 0.100692585}